In [25]:
# !ssh-keygen -t ed25519 -f ~/.ssh/id_ed25519 -N '' # generate my ssh key encryrpt it and store it . it will be accessed by an empty string
# !cat ~/.ssh/id_ed25519.pub # print the public key
# !chmod 700 ~/.ssh # i want secuity and personal access to it
# !ssh-keyscan github.com >> ~/.ssh/known_hosts # make recognise github
# !ssh -T git@github.com
# !git clone git@github.com:coded-by-49/Telecom-churn-model.git # cloning my repo into collab
# !git config --global user.name "coded-by-49" # setting up global identity
# !git config --global user.email "fabianbernard49@gmail.com"
# !git config --global --list | grep "user.name\|user.email" # checking whether my identity is setup
!cp /content/drive/MyDrive/ColabNotebooks/Tele_churn /content/Telecom-churn-model/



In [3]:

from itertools import groupby
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from IPython.display import clear_output
import tensorflow as tf
fc = tf.compat.v2.feature_column
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# GET OUR THE FILE READY
from google.colab import drive
import urllib
drive.mount('/content/drive')
tele_file = "/content/drive/MyDrive/GOOGLE COLLAB PROJECTS/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df_tele = pd.read_csv(tele_file)

# LEARNIN ALGORITHM
from sklearn.linear_model import LogisticRegression
df_tele.drop("customerID", axis = 1, inplace = True) # removed this unimportant feature

# CLEANING OF DATA
df_tele["TotalCharges"] = df_tele["TotalCharges"].replace(" ", np.nan) # replacing empty strings np.nan, to that i can convert it to float
df_tele.dropna(subset = ["TotalCharges"], inplace=True) # cleaded out the array by dropping does empty values within the Totalcharges feature
df_tele["TotalCharges"] = pd.to_numeric(df_tele["TotalCharges"]) # coverted to float because of the kind of content within totalcharges

# ENCODING OF ALL CATEGORICAL FEATURES
encoded = pd.get_dummies(df_tele, drop_first= True ) # this is one-hot key encoding (we want to transform all our categorical columns into )
encoded.head()

# CHANGING THE DATATYPE OF BINARY CATEGORICAL FEATURES
bool_cols = encoded.select_dtypes(include = "bool").columns #select all columns with bool datatype
encoded[bool_cols] = encoded[bool_cols].astype(int) #change all of them to int (0 and 1)

encoded["tenure"] = encoded["tenure"].astype(float)
all_numerical_features = encoded.select_dtypes(include = np.number).columns.tolist()

needed_numerical_feature = [feature for feature in all_numerical_features if encoded[feature].nunique()>2] # getting the numerical feature titles ready

Labels = encoded["Churn_Yes"]
Features = encoded.drop("Churn_Yes", axis = 1)

# OBTANING OUR TRANING AND TEST DATA
X_train, X_test, Y_train, Y_test = train_test_split(Features,Labels, test_size = 0.2, random_state = 42 )
# SCALLING AND PREPARING OUR TRAINING AND TESTING DATA
Scaler = StandardScaler()
# let us fit and tranform  our training numerical-features
X_train[needed_numerical_feature] = Scaler.fit_transform(X_train[needed_numerical_feature])

# let us apply the scaling parameters gotten from above and fit it to our testing numerical-features
X_test[needed_numerical_feature] = Scaler.transform(X_test[needed_numerical_feature])

log_regression = LogisticRegression() # is max_iter really needed?
log_regression.fit(X_train, Y_train) # FIT THIS WHERE OUR WEIGHT AND BIAS ARE DERVIED FROM

print(f"Training Accuracy: {log_regression.score(X_train, Y_train)}")
print(f"Testing Accuracy: {log_regression.score(X_test, Y_test)}")


# LOGISTC REGRESSION FROM SCRATCH

class Logistic_regression:
  def __init__(self, learning_rate = 0.01, n_iters = 1000):
    self.learning_rate = learning_rate
    self.n_iters = n_iters

  def sigmoid(self,z):
    return 1/(1+np.exp(-z))

# This is responsible deriving weight and bias of a several feature(s)
  def fit (self,X,Y):
    self.num_samples, self.num_features = X.shape #lets count the number of features and dataitems(rows)
    self.weights = np.zeros(self.num_features)
    self.bias = 0

    for _ in range(self.n_iters):
      Y_num_prediction = np.dot(X, self.weights) + self.bias
      Y_predicted = self.sigmoid(Y_num_prediction)

      # implementing gradient descent

      # we are implementing the average our two derivatives of the loss function
      dw = np.dot((Y_predicted - Y),X.T)/self.num_samples
      db = np.mean((Y_predicted - Y))

      # adjust weights after each iteration with learning rate
      self.weights = self.weights - self.learning_rate * dw
      self.bias = self.bias - self.learning_rate * db

      # stoping condition with eculidean normalisation
      if np.linalg.norm(db) < 0.001 and np.linalg.norm(dw) < 0.001:
        break

  def predict(self, X, Y):
    Y_num_prediction = np.dot(X, self.weights) + self.bias
    Y_predicted = self.sigmoid(Y_num_prediction)
    return Y_predicted


MessageError: Error: credential propagation was unsuccessful